# Shipper-Brand Relation

Esse notebook tem por objetivo fazer a relação de empresas que importam somente uma marca, facilitando a relação importador-

In [ ]:
# Importing the modules needed
import sys
sys.path.append("../src/")

from src.data.dremio_utils import *
# Data Handling
from dotenv import dotenv_values 
from pyarrow import flight
import pandas as pd

In [ ]:
config = dotenv_values(".env")
bds = BaseDremioService(config)

## 1. Extract

In [ ]:
big_data_di_sql = ''' 
SELECT 
    *
FROM estatisticas_di
'''
aux_df = bds.extract_dremio_dataset(big_data_di_sql)

In [ ]:
aux_df.to_parquet("../data/raw/attributes_dict.parquet")

# 2. Transform the Data

In [59]:
df = aux_df.copy()

In [60]:
# Filtrar somente as marcas
def attributes_to_marca(row):
    dict_att = eval(row)
    
    val = "sem marca"
    if "MARCA" in dict_att.keys():
        val = dict_att["MARCA"]
    
    return val
df["MARCA"] = df["attributes_pt"].apply(attributes_to_marca)

In [61]:
df.drop_duplicates(subset=["shipper", "MARCA"], inplace = True)
df.drop(columns = "attributes_pt", inplace = True)
df = df[df["MARCA"]!="sem marca"]

In [64]:
df_group = df.groupby("shipper", as_index = False)["MARCA"].nunique()

In [72]:
shippers_with_one_brand = df_group[df_group["MARCA"]==1]["shipper"].to_list()

In [75]:
df_with_one_brand = df[df["shipper"].isin(shippers_with_one_brand)].reset_index(drop=True)
df_with_one_brand.to_parquet("../data/processed/shipper_brand_relation.parquet")

In [81]:
df_aux = df_with_one_brand.copy()
df_aux["brand_with_shipper_name"] = df_aux.apply(lambda x: 1 if x["MARCA"] in x["shipper"] else 0, axis=1)
brand_with_shipper_name = df_aux[df_aux["brand_with_shipper_name"] == 1].copy()
brand_with_shipper_name.drop(columns="brand_with_shipper_name")
brand_with_shipper_name.reset_index(drop=True)
brand_with_shipper_name.to_parquet("../data/processed/brands_with_shipper_name.parquet")